# YOLOv4

In [3]:
import os
import sys
import shutil
repo = 'PyTorch_YOLOv4'

# 檢查是否已有 PyTorch_YOLOv4 的 repo, 若有則刪除
if os.path.exists(repo):
    shutil.rmtree(repo)  # delete output folder
!git clone https://github.com/d246810g2000/PyTorch_YOLOv4.git
    
sys.path.append(repo)
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

Cloning into 'PyTorch_YOLOv4'...
remote: Enumerating objects: 817, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 817 (delta 68), reused 131 (delta 38), pack-reused 648
Receiving objects: 100% (817/817), 34.64 MiB | 12.10 MiB/s, done.
Resolving deltas: 100% (421/421), done.


### 安裝所需套件

In [2]:
!pip install wandb pycocotools -q

### 資料集轉換格式

In [4]:
import glob
import random
import xml.etree.ElementTree as ET

def getImagesInDir(dir_path):
    img_formats = ['bmp', 'jpg', 'jpeg', 'png', 'tif', 'tiff', 'dng']  # acceptable image suffixes
    image_list = []
    for img_format in img_formats:
        for filename in glob.glob(dir_path + f'/*.{img_format}'):
            image_list.append(filename)

    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(img_path, ann_dir, output_image_path, output_label_path):
    basename = os.path.basename(img_path)
    basename_no_ext = os.path.splitext(basename)[0]
    
    #copy image
    shutil.copy(img_path, os.path.join(output_image_path, basename))

    in_file = open(ann_dir + '/' + basename_no_ext + '.xml')
    out_file = open(output_label_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

name = 'pedestrian'
classes = ['pedestrian']
train_test_split_rate = 0.2

img_dir = 'person_reid_datasets/train/JPEGImages/'
ann_dir = 'person_reid_datasets/train/Annotations/'
image_paths = getImagesInDir(img_dir)
random.seed(2022)
random.shuffle(image_paths)

train_image_path = f'PyTorch_YOLOv4/datasets/{name}/train/images/'
train_label_path = f'PyTorch_YOLOv4/datasets/{name}/train/labels/'
valid_image_path = f'PyTorch_YOLOv4/datasets/{name}/valid/images/'
valid_label_path = f'PyTorch_YOLOv4/datasets/{name}/valid/labels/'

if not os.path.exists(train_image_path):
    os.makedirs(train_image_path)
if not os.path.exists(train_label_path):
    os.makedirs(train_label_path)
if not os.path.exists(valid_image_path):
    os.makedirs(valid_image_path)
if not os.path.exists(valid_label_path):
    os.makedirs(valid_label_path)

train_test_split = len(image_paths)*train_test_split_rate

for i, img_path in enumerate(image_paths):
    if i >= train_test_split:
        # train
        convert_annotation(img_path, ann_dir, train_image_path, train_label_path)
    else:
        # valid
        convert_annotation(img_path, ann_dir, valid_image_path, valid_label_path)

### 創建 yaml 和 cfg 檔

In [5]:
!echo -e 'train: PyTorch_YOLOv4/datasets/pedestrian/train/images\nval: PyTorch_YOLOv4/datasets/pedestrian/valid/images\n\nnc: 1\nnames: [pedestrian]' > PyTorch_YOLOv4/data/pedestrian.yaml
!head PyTorch_YOLOv4/data/pedestrian.yaml

train: PyTorch_YOLOv4/datasets/pedestrian/train/images
val: PyTorch_YOLOv4/datasets/pedestrian/valid/images

nc: 1
names: [pedestrian]


In [6]:
!cp PyTorch_YOLOv4/cfg/yolov4.cfg PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg
!sed -n -e 959p -e 966p -e 1046p -e 1053p -e 1133p -e 1140p PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg

filters=255
classes=80
filters=255
classes=80
filters=255
classes=80


In [7]:
!sed -i '959s/255/18/' PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg # (classes + 5)x3
!sed -i '966s/80/1/' PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg
!sed -i '1046s/255/18/' PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg # (classes + 5)x3
!sed -i '1053s/80/1/' PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg
!sed -i '1133s/255/18/' PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg # (classes + 5)x3
!sed -i '1140s/80/1/' PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg

In [8]:
!sed -n -e 959p -e 966p -e 1046p -e 1053p -e 1133p -e 1140p PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg

filters=18
classes=1
filters=18
classes=1
filters=18
classes=1


### 使用 wandb 記錄訓練過程
- 先進入官網登入：https://wandb.ai/site 
- 右上方頭像 -> settings -> 複製 API keys -> 填入下方輸入處

In [9]:
import wandb
# Logging
id = wandb.util.generate_id()
wandb_run = wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: d246810g2000 (use `wandb login --relogin` to force relogin)


In [10]:
id

'21s20fv5'

### 開始訓練

In [1]:
# command line training
!python PyTorch_YOLOv4/train.py --device 0 --batch-size 4 --img-size 416 --data PyTorch_YOLOv4/data/pedestrian.yaml --cfg PyTorch_YOLOv4/cfg/yolov4_pedestrian.cfg --weights '' --name $id --epochs 300 --multi-scale

/bin/bash: built-in: No such file or directory
